# Recipe for creating credential, retrieving externalID and creating IAM role with your own scripting


The only unique thing about each Cloudability CloudFormation template is the externalID itself. This is a AWS best practice from a security perspective, but instead of needing to generate a separate template each time you could just activate a credential, retrieve it's externalID and then script on your end to create the role. 

Below details what this script does:

1. Creates the credential within Cloudability
2. Returns externalID as a CSV output

Next:
3. Use a script on your end to apply the externalID as a parameter with your own template. For example using a lookup in Terraform or Ansible:

    Example for Ansible: https://docs.ansible.com/ansible/2.4/playbooks_lookups.html
    
    Example for Terraform: https://www.terraform.io/docs/configuration/functions/lookup.html
    
4. Verify Credentials for an account: https://github.com/chrisdicken/Cloudability/blob/main/Verify%20All%20AWS%20Accounts.ipynb


Cloudability Documentation: https://developers.cloudability.com/docs/vendor-credentials-end-point

In [1]:
#This script first pulls the list of your acccounts, then creates the credential within Cloudability, then pulls the list of external and account id's and exports in a friendly format
#This was written in Python on Jupyter Lab via the Aanaconda Distribution using the standard libraries
#It should be fairly simplisitc to translate to any Python CLI
#Note: this code is dependant on the Cloudability API return speed, mutli-threading will be added in the future to reduce runtime

#to get list of all External and Account ids enter your Cloudability API token below and run code

token = "example"

In [ ]:
import pandas as pd
import requests 
import base64
from pandas.io.json import json_normalize
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

########--Variables--########

#encode token for html
token2 = token+":"
message_bytes = token2.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')
#API details
url = 'https://api.cloudability.com/v3/vendors/aws/accounts'
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic '+base64_message
}

########--Functions--########

#query cloudability, returns dataframe (table)
def Query_Cloudability(url, headers, payload):
    response = requests.request("GET", url, headers=headers, data = payload)
    print("response code data pull: "+str(response.status_code))
    if "v3" in url:
        df = json_normalize(response.json(), record_path=['result'])
    else:
        df = json_normalize(response.json())
    return df

########--Code--########

#get list of account id's
df = Query_Cloudability(url, headers, '')

#Generate External ID for all accounts
for x in df["id"]:
    payload  = '{\r\n    \"vendorAccountId\":\"'+ x + '\",\r\n    \"type\": \"aws_role\"\r\n}'
    response = requests.request("POST", url, headers=headers, data = payload)
    print(x+" response code: "+str(response.status_code))

print("End")

#get list of account id's and external id's
output = Query_Cloudability(url, headers, '')

#Export list as CSV
External_ID_Accound_ID = output[["id","vendorAccountName","authorization.externalId"]]
External_ID_Accound_ID.columns = ["Account_ID", "Account_Name", "External_ID"]
External_ID_Accound_ID.to_csv("External_id_List.csv", header = "True", index= "False")